<a href="https://colab.research.google.com/github/861728/ReserveService/blob/main/%ED%9A%8C%EC%82%AC_%EB%84%A4%EC%9D%B4%EB%B2%84%EC%A7%80%EB%8F%84_%EB%A7%81%ED%81%AC_%ED%81%AC%EB%A1%A4%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. 기존 패키지 정리 및 정식 구글 크롬 설치 (코랩 최적화)
!apt-get update
!apt-get install -y wget gnupg
!wget -q -O - https://dl-ssl.google.com/linux/linux_signing_key.pub | apt-key add -
!sh -c 'echo "deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main" >> /etc/apt/sources.list.d/google-chrome.list'
!apt-get update
!apt-get install -y google-chrome-stable

# 셀레니움 최신 버전 설치
!pip install selenium

import gspread
import time
from google.colab import auth
from google.auth import default
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# 2. 구글 시트 인증
print("🔐 구글 인증 시작...")
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
print("✅ 인증 성공!")



Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [85.0 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://cli.github.com/packages stable/main amd64 Packages [355 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,904 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [24.6 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [6,529 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,756 

In [ ]:
# --- [사용자 설정 구역] ---
SHEET_NAME = '네이버 검색 자동화'
TAB_NAME = '매장 정보 최신화'
START_ROW = 1501
END_ROW = 1812
SAVE_INTERVAL = 10
# -----------------------

def get_driver():
    options = Options()
    # 최신 크롬은 --headless=new 옵션을 권장합니다.
    options.add_argument('--headless=new')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--window-size=1920,1080')
    options.add_argument('--blink-settings=imagesEnabled=false')
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

    # [핵심] 설치한 정식 크롬 브라우저의 경로를 지정
    options.binary_location = "/usr/bin/google-chrome"

    # 드라이버는 셀레니움 4.x의 '셀레니움 매니저'가 브라우저 버전에 맞춰 자동 다운로드합니다.
    return webdriver.Chrome(options=options)

try:
    # 3. 시트 열기
    sh = gc.open(SHEET_NAME)
    worksheet = sh.worksheet(TAB_NAME)
    all_names = worksheet.col_values(2) # B열

    driver = get_driver()
    update_buffer = []

    print(f"🚀 {START_ROW}번행부터 데이터 수집을 시작합니다.")

    for i in range(START_ROW, END_ROW + 1):
        if i > len(all_names): break
        name = all_names[i-1]

        if not name or name.strip() == "":
            update_buffer.append([""])
            continue

        try:
            # 네이버 지도 검색
            driver.get(f"https://map.naver.com/p/search/{name}")

            # 상세 페이지 자동 이동 대기
            try:
                WebDriverWait(driver, 6).until(EC.url_contains("/place/"))
            except:
                # 자동 이동 실패 시 리스트에서 첫 번째 항목 클릭
                try:
                    WebDriverWait(driver, 4).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "searchIframe")))

                    # 현재 네이버 지도에서 검색 리스트 항목을 찾는 최적화된 선택자
                    first_item = WebDriverWait(driver, 4).until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, "li[data-lst-item] .title_box, .L_P_Z, .place_bluelink"))
                    )
                    first_item.click()

                    driver.switch_to.default_content()
                    WebDriverWait(driver, 6).until(EC.url_contains("/place/"))
                except:
                    driver.switch_to.default_content()

            # URL 정제 및 저장
            curr_url = driver.current_url
            if "/place/" in curr_url:
                clean_url = curr_url.split('?')[0] if '?' in curr_url else curr_url
                update_buffer.append([clean_url])
                print(f"[{i}] ✅ {name}")
            else:
                update_buffer.append([""])
                print(f"[{i}] ⚠️ 제외(결과 모호): {name}")

        except Exception:
            update_buffer.append(["오류"])
            print(f"[{i}] ❌ 에러: {name}")

        # 중간 저장
        if len(update_buffer) >= SAVE_INTERVAL:
            start_cell = f"D{i - len(update_buffer) + 1}"
            end_cell = f"D{i}"
            worksheet.update(range_name=f"{start_cell}:{end_cell}", values=update_buffer)
            update_buffer = []
            print(f"💾 {i}행까지 저장 완료")

    # 잔여 데이터 저장
    if update_buffer:
        actual_last_row = i
        start_cell = f"D{actual_last_row - len(update_buffer) + 1}"
        end_cell = f"D{actual_last_row}"
        worksheet.update(range_name=f"{start_cell}:{end_cell}", values=update_buffer)

except Exception as main_e:
    print(f"🚨 중단 원인: {main_e}")
finally:
    if 'driver' in locals():
        driver.quit()
    print("🏁 모든 작업 종료")

🚀 1501번행부터 데이터 수집을 시작합니다.
[1501] ⚠️ 제외(결과 모호): 태동반점
[1502] ⚠️ 제외(결과 모호): 태양생고기
[1503] ✅ 토속촌 삼계탕
[1504] ⚠️ 제외(결과 모호): 표선우동가게
[1505] ⚠️ 제외(결과 모호): 표선칼국수
[1506] ⚠️ 제외(결과 모호): 하동식당
[1507] ⚠️ 제외(결과 모호): 하이탕마라탕
[1508] ⚠️ 제외(결과 모호): 학곡리막국수닭갈비
[1509] ⚠️ 제외(결과 모호): 한남부대찌개
[1510] ⚠️ 제외(결과 모호): 한양양곱창
💾 1510행까지 저장 완료
[1511] ⚠️ 제외(결과 모호): 함경면옥
[1512] ✅ 함박웃음순대국
[1513] ⚠️ 제외(결과 모호): 합천식당
[1514] ⚠️ 제외(결과 모호): 합천일류돼지국밥
[1515] ⚠️ 제외(결과 모호): 현암뚝방구이
[1516] ⚠️ 제외(결과 모호): 형제육회
[1517] ⚠️ 제외(결과 모호): 혜화도담
[1518] ⚠️ 제외(결과 모호): 호호박
[1519] ✅ 화육계
[1520] ⚠️ 제외(결과 모호): 후계동
💾 1520행까지 저장 완료
[1521] ⚠️ 제외(결과 모호): 가원
[1522] ✅ 그리지하우스
[1523] ⚠️ 제외(결과 모호): 금성관나주곰탕
[1524] ✅ 낭만부대찌개
[1525] ⚠️ 제외(결과 모호): 명동게장
[1526] ⚠️ 제외(결과 모호): 복돈이부추삼겹살
[1527] ⚠️ 제외(결과 모호): 섬집
[1528] ⚠️ 제외(결과 모호): 장군본가
[1529] ⚠️ 제외(결과 모호): 진천집생고기
[1530] ⚠️ 제외(결과 모호): 코랏
💾 1530행까지 저장 완료
[1531] ⚠️ 제외(결과 모호): 아리조나막창
[1532] ⚠️ 제외(결과 모호): 앞산큰골집
[1533] ⚠️ 제외(결과 모호): 어항로
[1534] ✅ 타케모토
[1535] ⚠️ 제외(결과 모호): 우육면관
[1536] ✅ 란주칼면
[1537] ✅ 심야식당 하리
[1538] ⚠️ 제외(결과 모호): 경원식당